In [ ]:
import pandas as pd
import requests
import re
import os
from datetime import datetime, timedelta, date
import glob

URL_GENERIC_PART = 'https://www.vendeeglobe.org/download-race-data/vendeeglobe_'
START_DATE_HOUR = datetime(2020, 11, 8, 12, 0,0)
END_DATE_HOUR = datetime(2021, 3, 5, 9, 0, 0)

### Téléchargement des classements

In [ ]:
def daterange(start_date, end_date):
    delta = timedelta(hours=1)
    while start_date < end_date:
        yield start_date
        start_date += delta

def generate_urls(url, start_date, end_date):
    urls = []
    for single_date in daterange(start_date, end_date):
        urls.append(url+re.sub(r"(\:|\-|)", '', str(single_date.strftime("%Y-%m-%d %H:%M:%S")).replace(' ','_'))+'.xlsx')
    return urls

In [ ]:
def download_from_url(url):
    resp = requests.get(url)
    if resp.status_code == 200 :
        with open(url[-32:], 'wb') as output:
            output.write(resp.content)

def download_all_files(url, start_date, end_date,):
    urls = generate_urls(URL_GENERIC_PART, START_DATE_HOUR, END_DATE_HOUR)
    for url in urls:
        if os.path.isfile(url[-32:]) is False:
            download_from_url(url)

download_all_files(URL_GENERIC_PART, START_DATE_HOUR, END_DATE_HOUR)

In [ ]:
import tempfile
from zipfile import ZipFile
import shutil
import os
from fnmatch import fnmatch
import re

def change_in_zip(file_name, name_filter, change):
    tempdir = tempfile.mkdtemp()
    try:
        tempname = os.path.join(tempdir, 'new.zip')
        with ZipFile(file_name, 'r') as r, ZipFile(tempname, 'w') as w:
            for item in r.infolist():
                data = r.read(item.filename)           
                data = change(data)
                w.writestr(item, data)
        shutil.move(tempname, file_name)
    finally:
        shutil.rmtree(tempdir)

In [ ]:
#xls = pandas.read_excel(r'vendeeglobe_20210305_080000.xlsx', engine='openpyxl')
#xls

df = pd.DataFrame()
files = glob.glob('*.xlsx')
files.sort()
for filename in files:
    change_in_zip(filename, name_filter='xl/styles.xml',change=lambda d: re.sub(b'xxid="\d*"', b"", d)) 
    xlsx = pd.read_excel(filename, engine='openpyxl')
    df = df.append(xlsx)
    
df.head(1000)

In [ ]:
pd.set_option('display.max_rows', None)
df